In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sparknpl/training.csv
/kaggle/input/sparknpl/testdata.csv


In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=871ad780f41988d60724bc3781b9f28a38374ac94d1f29b60da429322c972122
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=37d9741e61fd9f27549ad2e540099d61309fe79fd1262ffc626e072573e738ac
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark
Note: you may need to restart the kernel to use updated packages.


**Thêm thử viện và môn tả chi tiết thư viện đó làm gì ?**

In [4]:
import pandas as pd 
import seaborn as sns 

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = (
    SparkSession.builder.appName("ModelTraining")
    .config("spark.executor.memory", "10")
    .getOrCreate()
)
pd.options.display.max_columns = None
pd.options.display.max_rows = 250
pd.options.display.max_colwidth = 150
sns.set(color_codes=True)
#  tạo tên và lưu vô ram có 6g
schema = "polarity FLOAT, id LONG, date_time STRING, query STRING, user STRING, text STRING"

IN_PATH_TEST = "/kaggle/input/sparknpl/testdata.csv"
IN_PATH_TRAIN = "/kaggle/input/sparknpl/training.csv"
OUT_PATH_RAW = "RAW"

# polarity là độ phổ biến


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/16 09:54:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/16 09:54:55 ERROR SparkContext: Error initializing SparkContext.
org.apache.spark.SparkIllegalArgumentException: [INVALID_EXECUTOR_MEMORY] Executor memory 10 must be at least 471859200. Please increase executor memory using the --executor-memory option or "spark.executor.memory" in Spark configuration.
	at org.apache.spark.memory.UnifiedMemoryManager$.getMaxMemory(UnifiedMemoryManager.scala:230)
	at org.apache.spark.memory.UnifiedMemoryManager$.apply(UnifiedMemoryManager.scala:201)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:325)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:196)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:284)
	at org.apache.spark.SparkContext.<i

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkIllegalArgumentException: [INVALID_EXECUTOR_MEMORY] Executor memory 10 must be at least 471859200. Please increase executor memory using the --executor-memory option or "spark.executor.memory" in Spark configuration.
	at org.apache.spark.memory.UnifiedMemoryManager$.getMaxMemory(UnifiedMemoryManager.scala:230)
	at org.apache.spark.memory.UnifiedMemoryManager$.apply(UnifiedMemoryManager.scala:201)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:325)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:196)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:284)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:483)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeCo

In [ ]:
# tạo ra format cho data frame
spark_reader = spark.read.schema(schema)
# 
test_data = spark_reader.csv(IN_PATH_TEST, header=False).cache()
training_data = spark_reader.csv(IN_PATH_TRAIN, header=False).cache()

In [ ]:
#  show data test
test_data.filter(f.col("polarity")==4.0).show(5,truncate=False)

In [ ]:
# Tính toán xem  có bao nhiêu row ứng với mỗi nhãn
test_data.groupBy("polarity").count().toPandas()

In [ ]:
#  show data train
training_data.show(5,truncate=False)

In [ ]:
import html
schema = "polarity FLOAT, id LONG, date_time TIMESTAMP, query STRING, user STRING, text STRING"
timestampformat = "EEE MMM dd HH:mm:ss zzz yyyy"
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
IN_PATH_RAW = "/kaggle/input/sparknpl/testdata.csv"
IN_PATH_TRAIN = "/kaggle/input/sparknpl/training.csv"
#OUT_PATH_CLEAN = "CLEAN"

spark_reader = spark.read.schema(schema)

user_regex = r"(@\w{1,15})"
hashtag_regex = "(#\w{1,})"
url_regex=r"((https?|ftp|file):\/{2,3})+([-\w+&@#/%=~|$?!:,.]*)|(www.)+([-\w+&@#/%=~|$?!:,.]*)"
email_regex=r"[\w.-]+@[\w.-]+\.[a-zA-Z]{1,}"


@f.udf
def html_unescape(s: str):
    if isinstance(s, str):
        return html.unescape(s)
    return s


def clean_data(df):
    df = (
        df
        .withColumn("original_text", f.col("text"))
        .withColumn("text", f.regexp_replace(f.col("text"), url_regex, ""))
        .withColumn("text", f.regexp_replace(f.col("text"), email_regex, ""))
        .withColumn("text", f.regexp_replace(f.col("text"), user_regex, ""))
        .withColumn("text", f.regexp_replace(f.col("text"), "#", " "))
        .withColumn("text", html_unescape(f.col("text")))
        .filter("text != ''")
    )
    return df

df_train_raw = spark_reader.csv(IN_PATH_RAW, timestampFormat=timestampformat) 
df_train_clean = clean_data(df_train_raw)
df_test_raw = spark_reader.csv(IN_PATH_TEST, timestampFormat=timestampformat) 
df_test_clean = clean_data(df_test_raw)

In [ ]:
df_test_clean_binary = clean_data(df_test_raw_binary)

In [ ]:
# in ra data đã clean
df_test_raw_binary = df_test_clean.filter(f.col("polarity")!= 2.0)
df_test_raw_binary.show(5)

In [ ]:
# test data clean not filter
df_train_clean.show(10,True)
df_test_clean.show(10,True)

In [ ]:
#  đếm độ phổ biến
df_test_clean.groupBy("polarity").count().toPandas()

**Lấy dữ liệu cẩn thiết để Train**

In [ ]:
traindf = (
    df_train_clean
    # Remove all numbers
    .withColumn("text", f.regexp_replace(f.col("text"), "[^a-zA-Z']", " "))
    # Remove all double/multiple spaces
    .withColumn("text", f.regexp_replace(f.col("text"), " +", " "))
    # Remove leading and trailing whitespaces
    .withColumn("text", f.trim(f.col("text")))
    # Ensure we don't end up with empty rows
    .filter("text != ''")
)

traindata = traindf.select("text", "polarity").coalesce(1).cache()

In [ ]:
#  show data
traindata.show(5,truncate =False),

In [ ]:
df_test = (
    df_test_clean
    # Remove all numbers
    .withColumn("text", f.regexp_replace(f.col("text"), "[^a-zA-Z']", " "))
    # Remove all double/multiple spaces
    .withColumn("text", f.regexp_replace(f.col("text"), " +", " "))
    # Remove leading and trailing whitespaces
    .withColumn("text", f.trim(f.col("text")))
    # Ensure we don't end up with empty rows
    .filter("text != ''")
)

testdata = df_test.select("text", "polarity").coalesce(1).cache()

In [ ]:
testdata.show(5,truncate= False )

In [ ]:
df_test_binary = (
    df_test_clean_binary
    # Remove all numbers
    .withColumn("text", f.regexp_replace(f.col("text"), "[^a-zA-Z']", " "))
    # Remove all double/multiple spaces
    .withColumn("text", f.regexp_replace(f.col("text"), " +", " "))
    # Remove leading and trailing whitespaces
    .withColumn("text", f.trim(f.col("text")))
    # Ensure we don't end up with empty rows
    .filter("text != ''")
)

testdata_binary = df_test_binary.select("text", "polarity").coalesce(1).cache()

In [ ]:
df_test.toPandas()

In [ ]:
#  Convert data thành để chuẩn bị test HashingTF
import time
from pyspark.ml.feature import (
    StopWordsRemover,
    Tokenizer,
    HashingTF,
    IDF,
)
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="text", outputCol="words1")
stopwords_remover = StopWordsRemover(
    inputCol="words1",
    outputCol="words2",
    stopWords=StopWordsRemover.loadDefaultStopWords("english")
)
hashing_tf = HashingTF(
    inputCol="words2",
    outputCol="term_frequency",
)
idf = IDF(
    inputCol="term_frequency",
    outputCol="features",
    minDocFreq=5,
)
lr = LogisticRegression(labelCol="polarity")

In [ ]:
#Tạo mô hình sử dụng pipeline
semantic_analysis_pipeline = Pipeline(
    stages=[tokenizer, stopwords_remover, hashing_tf, idf, lr]
)

semantic_analysis_model = semantic_analysis_pipeline.fit(traindata)

In [ ]:
#Tạo mô hình sử dụng pipeline
(traindata1, traindata2) = traindata.randomSplit([0.3, 0.7], seed=2020)

semantic_analysis_pipeline = Pipeline(
    stages=[tokenizer, stopwords_remover, hashing_tf, idf, lr]
)
semantic_analysis_model = semantic_analysis_pipeline.fit(traindata1)

In [ ]:
semantic_analysis_model.transform(testdata).show()


In [ ]:
#lưu lại model
semantic_analysis_model.save("NLP_Model_TW")

In [ ]:
semantic_analysis_model.transform(testdata).select("polarity","probability","prediction").show(50,False)


In [ ]:
from pyspark.ml import PipelineModel
trainedmodel = PipelineModel.load("/kaggle/input/spark-mllib-nlp/NLP_Model")